In [17]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
from torchvision.transforms import Resize, ToTensor, Normalize
import matplotlib.pyplot as plt
from imblearn.under_sampling import RandomUnderSampler
import cv2
from scipy import stats
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, roc_auc_score, \
    average_precision_score
from sklearn.model_selection import train_test_split
import time
import os
from pathlib import Path
from skimage import io
import copy
from torch import optim, cuda
import pandas as pd
import glob
from collections import Counter
# Useful for examining network
from functools import reduce
from operator import __add__
# from torchsummary import summary
import seaborn as sns
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
from PIL import Image
from timeit import default_timer as timer
import matplotlib.pyplot as plt

# Useful for examining network
from functools import reduce
from operator import __add__
from torchsummary import summary

# from IPython.core.interactiveshell import InteractiveShell
import seaborn as sns

import warnings
# warnings.filterwarnings('ignore', category=FutureWarning)

# Image manipulations
from PIL import Image

# Timing utility
from timeit import default_timer as timer

# Visualizations
import matplotlib.pyplot as plt


In [18]:
# train_df = pd.read_csv('/data/proj/cardiac-amyloid/ValveNet/data/57k_train_df_pace_removed_poor_quality_removed_any_patient_negative_as_tabular_metadata.csv')
# eval_df = pd.read_csv('/data/proj/cardiac-amyloid/ValveNet/data/29k_eval_df_pace_removed_poor_quality_removed_any_patient_negative_as_tabular_metadata.csv')
train_df = pd.read_csv('/data/proj/cardiac-amyloid/ValveNet/data/57k_train_df_pace_removed_2x_poor_quality_removed_any_patient_negative_as_tabular_metadata.csv')
eval_df = pd.read_csv('/data/proj/cardiac-amyloid/ValveNet/data/29k_eval_df_pace_removed_2x_poor_quality_removed_any_patient_negative_as_tabular_metadata.csv')


In [19]:
def balance_binary(df, label, sampling_strategy=1):
    y = df[label]
    X = df
    binary_mask = np.bitwise_or(y == 0, y == 1)
    binary_y = y[binary_mask]
    binary_X = X[binary_mask]
    rus = RandomUnderSampler(sampling_strategy=sampling_strategy)
    X_res, y_res = rus.fit_resample(binary_X, binary_y)
    return X_res

In [26]:
# train_available_path = Path("/data/proj/cardiac-amyloid/ValveNet/data/50k_train_df_pace_removed_any_patient_negative_available_for_GCP_server.csv")
train_available_path = Path("/data/proj/cardiac-amyloid/ValveNet/data/57k_train_df_pace_removed_2x_poor_quality_removed_any_patient_negative_as_tabular_metadata.csv")
#TODO this only works if this file path doesnt already exist
print("make train_df_available", time.process_time()/100000,'seconds')
if not Path(train_available_path).exists():
    print("No train_df_available found, creating now...")
    npyfilespath = "/data/proj/cardiac-amyloid/ekg_waveforms_output/"
    os.chdir(npyfilespath)
    npfiles = glob.glob("*.npy")
    npfiles.sort()
    npdf = pd.DataFrame({'filename': npfiles})
    train_df_available = train_df.merge(npdf,left_on='filename', right_on='filename', validate='1:1')
    eval_df_available = eval_df.merge(npdf,left_on='filename', right_on='filename', validate='1:1')

    # print("balance datasets", time.process_time())
    label = "stenosis_label_binary"
    # train_df_available = balance_binary(train_df_available, label)
    # eval_df_available = balance_binary(eval_df_available,label)
    train_df_available = train_df_available
    eval_df_available = eval_df_available
    train_df_available.to_csv("/data/proj/cardiac-amyloid/ValveNet/data/57k_train_df_pace_removed_2x_poor_quality_removed_any_patient_negative_as_tabular_metadata.csv")
    eval_df_available.to_csv("/data/proj/cardiac-amyloid/ValveNet/data/29k_eval_df_pace_removed_2x_poor_quality_removed_any_patient_negative_as_tabular_metadata.csv")
else:
    print('train_df_available found, passing in train_df')
    train_df_available = train_df
    eval_df_available = eval_df

make train_df_available 0.14609136083105 seconds
train_df_available found, passing in train_df


In [27]:
filelist_train = train_df_available['filename']
filelist_eval = eval_df_available['filename']

In [28]:
print('train DF length:', len(train_df), '\n eval DF length:', len(eval_df))
print('train DF available length:', len(filelist_train), '\n eval DF available length:', len(filelist_eval))
print('train ECG count:', len(train_df_available), '\n eval ECG count:', len(eval_df_available))

train DF length: 55593 
 eval DF length: 27984
train DF available length: 55593 
 eval DF available length: 27984
train ECG count: 55593 
 eval ECG count: 27984


In [29]:
npyfilespath = "/data/proj/cardiac-amyloid/ekg_waveforms_output/"
fpath_train = "/data/proj/cardiac-amyloid/ValveNet/data/57k_train_df_pace_removed_2x_poor_quality_removed_any_patient_negative_as_tabular_metadata.npy"
fpath_eval = "/data/proj/cardiac-amyloid/ValveNet/data/29k_eval_df_pace_removed_2x_poor_quality_removed_any_patient_negative_as_tabular_metadata.npy"

In [30]:
try:
    trainData = np.load(fpath_train)  # Load input data. Input data should be compiled as numpy arrays with (sample numbers, time, lead, 1)
    evalData = np.load(fpath_eval)
    print('Successfully loaded pre-built trainData and evalData')
except:
    trainData = []
    evalData = []

time1 = timer()
if not Path(fpath_train).exists():  # or (len(trainData)!=len(train_df_available)) or (len(evalData)!=len(eval_df_available)):
    print("no train array found, building now...")
    for npfile in filelist_train:
        i = 0
        try:
            path = os.path.join(npyfilespath + npfile)
            file = np.load(path)
            trainData.append(file)
            i += 1
            if i % 1 == 100:
                print("{i} EKGs have been written to array")
        except:
            continue
    trainData = np.array(trainData)
    np.save(fpath_train, trainData)

if not Path(fpath_eval).exists():  # or (len(trainData)!=len(train_df_available)) or (len(evalData)!=len(eval_df_available)):
    print("no eval array found, building now...")
    for npfile in filelist_eval:
        i = 0
        try:
            path = os.path.join(npyfilespath + npfile)
            file = np.load(path)
            evalData.append(file)
            i += 1
            if i % 1 == 100:
                print("{i} EKGs have been written to array")
        except:
            continue
    evalData = np.array(evalData)
    np.save(fpath_eval, evalData)
print("time to generate arrays: ", timer() - time1)

Successfully loaded pre-built trainData and evalData
time to generate arrays:  0.00041944533586502075


In [31]:
trainData.shape

(55593, 2500, 12, 1)

In [32]:
unprocessed_train = np.transpose(trainData, axes=[0, 3, 1, 2])
print('unprocessed_train shape for model:', unprocessed_train.shape)
unprocessed_eval = np.transpose(evalData, axes=[0, 3, 1, 2])
print('unprocessed_eval shape for model:', unprocessed_eval.shape)
assert unprocessed_train.shape[1:] == (1, 2500, 12), "train is not X,1,2500,12"
assert unprocessed_eval.shape[1:] == (1, 2500, 12), "eval is not X,1,2500,12"

# Write out feature arrays unprocessed if desired
# np.save('/data/proj/cardiac-amyloid/ValveNet/data/57k_train_df_pace_removed_2x_poor_quality_removed_any_patient_negative_as_features_unprocessed.npy', unprocessed_train)
# np.save('/data/proj/cardiac-amyloid/ValveNet/data/29k_eval_df_pace_removed_2x_poor_quality_removed_any_patient_negative_as_features_unprocessed.npy', unprocessed_eval)


unprocessed_train shape for model: (55593, 1, 2500, 12)
unprocessed_eval shape for model: (27984, 1, 2500, 12)


In [33]:
# # Transpose for normalization
'''02/06/21 update, truncate_data_median now does per-lead truncation.
The median and MAD are recaluclated after truncation is complete 
and eval is normalized before train, neither should make significant differences but are more correct'''

train_data = np.transpose(trainData, axes=[0, 2, 1, 3])
print('train_data norm_prep shape:', train_data.shape)
eval_data = np.transpose(evalData, axes=[0, 2, 1, 3])
print('eval_data norm_prep shape:', eval_data.shape)
assert train_data.shape[1:] == (12, 2500, 1), "train is not X,12,2500,1"
assert eval_data.shape[1:] == (12, 2500, 1), "eval is not X,12,2500,1"

sigma=5

def truncate_data_median(target_array, median, mad):
    datamadlimit = target_array
    assert (len(med_tr) == datamadlimit.shape[1])
    # Truncate data to sigma*mad to limit outlier effect and use robust statistical approach over mean
    for i in range(len(median)):
        print("pre-truncation min:",datamadlimit[:, i, :, :].min())
        print("pre-truncation max:", datamadlimit[:, i, :, :].max())
        datamadlimit[:, i, :, :] = np.where(datamadlimit[:, i, :, :] > median[i] + (sigma * mad[i]), median[i] + (sigma * mad[i]), datamadlimit[:, i, :, :])
        datamadlimit[:, i, :, :] = np.where(datamadlimit[:, i, :, :] < median[i] - (sigma * mad[i]), median[i] - (sigma * mad[i]), datamadlimit[:, i, :, :])
    for i in range(len(median)):
        print("post-truncation min:",datamadlimit[:, i, :, :].min())
        print("post-truncation max:", datamadlimit[:, i, :, :].max())
    # datanorm2 = cv2.normalize(datamadlimit, datamadlimit, -1, 1, cv2.NORM_MINMAX)
    return datamadlimit


# Compute mean and standard deviation for normalization
# median = np.median(train_data)
# mad = stats.median_absolute_deviation(train_data)
mean_norm, std_norm = np.mean(train_data, axis=(0, 2, 3)), np.std(train_data, axis=(0, 2, 3))
print('Mean and STD before truncation', mean_norm, std_norm)
med_tr, mad_tr = np.median(train_data, axis=(0, 2, 3)), stats.median_absolute_deviation(train_data, axis=(0, 2, 3))
print("Median and MAD before truncation", med_tr, mad_tr)

# NOTE: NEED TO put eval_data before train_data to avoid a bug wherein we modify train_data and median/mad
eval_data = truncate_data_median(eval_data, med_tr, mad_tr)
print("eval data truncated based on median")
train_data = truncate_data_median(train_data, med_tr, mad_tr)
print("train data truncated based on median")

mean_norm, std_norm = np.mean(train_data, axis=(0, 2, 3)), np.std(train_data, axis=(0, 2, 3))
print('Mean and STD before normalization', mean_norm, std_norm)
med_tr, mad_tr = np.median(train_data, axis=(0, 2, 3)), stats.median_absolute_deviation(train_data, axis=(0, 2, 3))
print("Median and MAD before normalization", med_tr, mad_tr)

# Per-lead normalization (there are 12 leads)
assert (len(med_tr) == train_data.shape[1])
for i in range(len(med_tr)):
    tic = time.perf_counter()
    print("pre-normalizing min:",train_data[:, i, :, :].min())
    print("pre-normalizing max:", train_data[:, i, :, :].max())
    eval_data[:, i, :, :] = (eval_data[:, i, :, :] - med_tr[i]) / mad_tr[i]
    train_data[:, i, :, :] = (train_data[:, i, :, :] - med_tr[i]) / mad_tr[i]
    print("post-normalizing min:",train_data[:, i, :, :].min())
    print("post-normalizing max:", train_data[:, i, :, :].max())
    toc = time.perf_counter()
    print(f'Processed iter {i} in {toc - tic:0.2f} seconds')

mean_norm, std_norm = np.mean(train_data, axis=(0, 2, 3)), np.std(train_data, axis=(0, 2, 3))
print('Mean and STD after normalization', mean_norm, std_norm)
med_norm, mad_norm = np.median(train_data, axis=(0, 2, 3)), stats.median_absolute_deviation(train_data, axis=(0, 2, 3))
print("Median and MAD after normalization", med_norm, mad_norm)

# Transpose data for model
X_train = np.transpose(train_data, axes=[0, 3, 2, 1])
print('X_train shape for model:', X_train.shape)
X_eval = np.transpose(eval_data, axes=[0, 3, 2, 1])
print('X_eval shape for model:', X_eval.shape)
assert X_train.shape[1:] == (1, 2500, 12), "train is not X,1,2500,12"
assert X_eval.shape[1:] == (1, 2500, 12), "eval is not X,1,2500,12"



train_data norm_prep shape: (55593, 12, 2500, 1)
eval_data norm_prep shape: (27984, 12, 2500, 1)
Mean and STD before truncation [-0.57058358 -0.52162918  0.04895439  0.54610638 -0.0244772   0.0244772
 -0.65568828 -0.68845719 -0.79558483 -0.67047087 -0.66145356 -0.67744864] [36.69501863 34.69142697 38.8391823  29.96479983 19.41959115 19.41959115
 47.15307279 66.56324286 72.08039239 59.55403582 52.06029694 46.56159721]
Median and MAD before truncation [-5.  -4.   2.   4.5 -1.   1.   4.   2.   1.  -2.  -4.  -5. ] [11.8608 13.3434 11.8608 11.1195  5.9304  5.9304 13.3434 17.7912 19.2738
 17.7912 16.3086 14.826 ]
pre-truncation min: -2019.0
pre-truncation max: 1294.0
pre-truncation min: -7428.0
pre-truncation max: 4314.0
pre-truncation min: -7352.0
pre-truncation max: 4214.0
pre-truncation min: -2207.0
pre-truncation max: 3752.0
pre-truncation min: -2107.0
pre-truncation max: 3676.0
pre-truncation min: -3676.0
pre-truncation max: 2107.0
pre-truncation min: -32188.0
pre-truncation max: 31514.

In [36]:
# Write out feature arrays
np.save('/labs/amyloid/CardiacAmyloid/ValveNetExtraData/57k_train_df_pace_removed_2x_poor_quality_removed_any_patient_negative_as_features_5_sigma_robust_normalization.npy', X_train)
np.save('/labs/amyloid/CardiacAmyloid/ValveNetExtraData/29k_eval_df_pace_removed_2x_poor_quality_removed_any_patient_negative_as_features_5_sigma_robust_normalization.npy', X_eval)
print('Saved feature arrays')

Saved feature arrays


In [ ]:

#Commented out below is old approach pre 2/06/21
# # Transpose for normalization
# train_data = np.transpose(trainData, axes=[0, 2, 1, 3])
# print('train_data norm_prep shape:', train_data.shape)
# eval_data = np.transpose(evalData, axes=[0, 2, 1, 3])
# print('eval_data norm_prep shape:', eval_data.shape)
# assert train_data.shape[1:] == (12, 2500, 1), "train is not X,12,2500,1"
# assert eval_data.shape[1:] == (12, 2500, 1), "eval is not X,12,2500,1"

# sigma=5

# def truncate_data_median(target_array, median, mad):
#     datamadlimit = target_array
#     assert (len(med_tr) == datamadlimit.shape[1])
#     # Truncate data to sigma*mad to limit outlier effect and use robust statistical approach over mean
#     for i in range(len(median)):
#         print("pre-truncation min:",datamadlimit[:, i, :, :].min())
#         print("pre-truncation max:", datamadlimit[:, i, :, :].max())
#         datamadlimit[:, i, :, :] = np.where(datamadlimit[:, i, :, :] > median[i] + (sigma * mad[i]), median[i] + (sigma * mad), datamadlimit[:, i, :, :])
#         datamadlimit[:, i, :, :] = np.where(datamadlimit[:, i, :, :] < median[i] - (sigma * mad[i]), median[i] - (sigma * mad), datamadlimit[:, i, :, :])
#     for i in range(len(median)):
#         print("post-truncation min:",datamadlimit[:, i, :, :].min())
#         print("post-truncation max:", datamadlimit[:, i, :, :].max())
#     # datanorm2 = cv2.normalize(datamadlimit, datamadlimit, -1, 1, cv2.NORM_MINMAX)
#     return datamadlimit


# # Compute mean and standard deviation for normalization
# # median = np.median(train_data)
# # mad = stats.median_absolute_deviation(train_data)
# mean_norm, std_norm = np.mean(train_data, axis=(0, 2, 3)), np.std(train_data, axis=(0, 2, 3))
# print('Mean and STD before normalization', mean_norm, std_norm)
# med_tr, mad_tr = np.median(train_data, axis=(0, 2, 3)), stats.median_absolute_deviation(train_data, axis=(0, 2, 3))
# print("Median and MAD before normalization", med_tr, mad_tr)

# # NOTE: NEED TO put eval_data before train_data to avoid a bug wherein we modify train_data and median/mad
# eval_data = truncate_data_median(eval_data, med_tr, mad_tr)
# print("eval data truncated based on median")
# train_data = truncate_data_median(train_data, med_tr, mad_tr)
# print("train data truncated based on median")

# # Per-lead normalization (there are 12 leads)
# assert (len(med_tr) == train_data.shape[1])
# for i in range(len(med_tr)):
#     tic = time.perf_counter()
#     print("pre-normalizing min:",train_data[:, i, :, :].min())
#     print("pre-normalizing max:", train_data[:, i, :, :].max())
#     train_data[:, i, :, :] = (train_data[:, i, :, :] - med_tr[i]) / mad_tr[i]
#     eval_data[:, i, :, :] = (eval_data[:, i, :, :] - med_tr[i]) / mad_tr[i]
#     print("post-normalizing min:",train_data[:, i, :, :].min())
#     print("post-normalizing max:", train_data[:, i, :, :].max())
#     toc = time.perf_counter()
#     print(f'Processed iter {i} in {toc - tic:0.2f} seconds')

# mean_norm, std_norm = np.mean(train_data, axis=(0, 2, 3)), np.std(train_data, axis=(0, 2, 3))
# print('Mean and STD after normalization', mean_norm, std_norm)
# med_norm, mad_norm = np.median(train_data, axis=(0, 2, 3)), stats.median_absolute_deviation(train_data, axis=(0, 2, 3))
# print("Median and MAD after normalization", med_norm, mad_norm)

# # Transpose data for model
# X_train = np.transpose(train_data, axes=[0, 3, 2, 1])
# print('X_train shape for model:', X_train.shape)
# X_eval = np.transpose(eval_data, axes=[0, 3, 2, 1])
# print('X_eval shape for model:', X_eval.shape)
# assert X_train.shape[1:] == (1, 2500, 12), "train is not X,1,2500,12"
# assert X_eval.shape[1:] == (1, 2500, 12), "eval is not X,1,2500,12"

